In [1]:
!pip install allantools

In [2]:
#!pip install datashader

In [3]:
#import datashader as ds

In [4]:
#import datashader.transfer_functions as tf

In [5]:
import random
import pandas as pd
import numpy as np
from allantools import adev
import time
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

# Sampling Rate 1kHz for 1 hour

In [6]:
fs = 10_000 #samples/second

In [7]:
#1 hour of sampling
ts = 3600 #sampling time in seconds

In [8]:
nSamples = fs*ts
nSamples

36000000

In [9]:
def generate_1k_samples():
    return np.array([random.gauss(0, 1) for i in range(0, 1000)]).astype("float128")

In [10]:
tic = time.perf_counter()
foo = generate_1k_samples()
toc = time.perf_counter()
tdelta = toc-tic
print(f"{tdelta:0.4f}")

0.0015


In [11]:
#estimated time for all samples to be generated
def estimate_completion(tdelta, nSamples):
    return f"Estimated {tdelta*(nSamples/1000)} sec to complete" 

In [12]:
estimate_completion(tdelta, nSamples)

'Estimated 52.64949601769331 sec to complete'

In [13]:
#generate 1000 samples nSamples/1000 times
dataset = np.array([generate_1k_samples() for i in range(int(nSamples/1000))]).astype("float128")

In [14]:
taus = np.array([adev(x)[0] for x in dataset])

In [15]:
ad = np.array([adev(x)[1].astype(float) for x in dataset])

In [16]:
flat_taus = taus.flatten()

In [17]:
flat_ad = ad.flatten()

In [18]:
df = pd.DataFrame(data={"Taus":flat_taus, "Allan Deviations":flat_ad})

In [19]:
#x_range = (df["Taus"].iloc[0], df["Taus"].iloc[-1]) 
#y_range = (df["Allan Deviations"].min(), df["Allan Deviations"].max())

In [20]:
#x_range, y_range

In [21]:
#tf.shade(ds.Canvas(x_range=x_range, y_range=y_range, x_axis_type="log", y_axis_type="log").points(df, "Taus", "Allan Deviations"))

In [22]:
nPoints = df["Allan Deviations"].shape[0]
nPoints

324000

In [23]:
plt.plot(np.arange(0, nSamples), dataset.flatten())
plt.savefig("WHITE_NOISE.png")
plt.close()

In [24]:
#plt.loglog(df["Taus"], df["Allan Deviations"])

#plt.savefig("TEST_ADEV.png")

In [25]:
tic = time.perf_counter()
(taus, ad, _, _) = adev(dataset.flatten())
toc = time.perf_counter()
print(f"{toc-tic:0.4f} seconds")

2.0590 seconds


In [26]:
import allantools
x = allantools.noise.white(nSamples)        # Generate some phase data, in seconds.
(taus, adevs, errors, ns) = allantools.adev(x, rate=fs)

In [27]:
plt.plot(np.arange(0, len(x)), x)
plt.savefig("ALLAN_TOOLS_WHITE_NOISE.png")

In [28]:
taus

array([1.000000e-04, 2.000000e-04, 4.000000e-04, 8.000000e-04,
       1.600000e-03, 3.200000e-03, 6.400000e-03, 1.280000e-02,
       2.560000e-02, 5.120000e-02, 1.024000e-01, 2.048000e-01,
       4.096000e-01, 8.192000e-01, 1.638400e+00, 3.276800e+00,
       6.553600e+00, 1.310720e+01, 2.621440e+01, 5.242880e+01,
       1.048576e+02, 2.097152e+02, 4.194304e+02, 8.388608e+02])

In [29]:
adevs

array([1.22459993e+04, 6.12093183e+03, 3.06310884e+03, 1.53201632e+03,
       7.65847274e+02, 3.82195152e+02, 1.91364664e+02, 9.55043563e+01,
       4.77213158e+01, 2.40465843e+01, 1.20726074e+01, 5.98876024e+00,
       2.98693522e+00, 1.49354132e+00, 7.62087317e-01, 3.61388997e-01,
       1.92436848e-01, 9.13771617e-02, 4.89542836e-02, 1.99900601e-02,
       1.03363813e-02, 4.27639301e-03, 2.94812879e-03, 1.68764913e-03])

In [32]:
df = pd.DataFrame(data={"AD":adevs})
df.describe()

,AD
count,24.000000
mean,1020.532206
std,2757.958734
min,0.001688
25%,0.080771
50%,4.487848
75%,239.072286
max,12245.999273


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AD      24 non-null     float64
dtypes: float64(1)
memory usage: 320.0 bytes


In [34]:
df.iloc[0].values

array([12245.99927283])

In [30]:
plt.close()

x = np.arange(1, len(taus))
y = 1/np.sqrt(x)
w = 1/x

plt.loglog(x, y, label="SLOPE -1/2")
plt.loglog(x, w, label="SLOPE -1")
plt.loglog(taus, adevs, label="ADEV")
plt.legend()
plt.savefig("STRAIGHT_LINES.png")